In [36]:
# make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path

file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))

%load_ext autoreload
%autoreload

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

sns.set()

from collections import namedtuple

from pa_lib.file import (
    project_dir,
    load_bin,
    write_xlsx,
    load_xlsx,
    store_bin,
)
from pa_lib.data import as_dtype, dtFactor, desc_col, lookup, clean_up_categoricals
from pa_lib.util import (
    collect,
    value,
    flatten,
    normalize_rows,
    normalize_cols,
    list_items,
    cap_words,
)
from pa_lib.log import time_log

# display long columns completely, show more rows
pd.set_option("display.max_colwidth", 300)
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
with project_dir("axinova"):
    ax_data = load_bin("ax_data.feather")

with project_dir("spr_plus"):
    spr_data = (
        load_xlsx("2019_11_13_VSTD_DIGITAL_2019_2020_APG.xlsx", sheet_name="Daten")
        .melt(
            id_vars="WT FLAECHE_ID ANBIETER SPR_FLAECHE_ID".split(),
            var_name="Time",
            value_name="Count",
        )
        .pipe(as_dtype, dtFactor, incl_dtype="object")
        .rename(mapper=lambda name: cap_words(name, sep="_"), axis="columns")
    )

    rail_displays = (
        load_xlsx("flaechen_rail_epanel.xlsx")
        .append(load_xlsx("flaechen_rail_eboard.xlsx"), ignore_index=True, sort=False)
        .pipe(as_dtype, dtFactor, incl_dtype="object")
    )

14:59:03 [INFO] Started loading binary file ...
14:59:03 [INFO] Reading from file C:\Users\kpf\data\axinova\ax_data.feather
14:59:03 [INFO] ... finished loading binary file in 0.22s (0.77s CPU)
14:59:03 [INFO] Started loading xlsx file ...
14:59:03 [INFO] Reading from file C:\Users\kpf\data\spr_plus\2019_11_13_VSTD_DIGITAL_2019_2020_APG.xlsx
14:59:06 [INFO] ... finished loading xlsx file in 2.57s (2.58s CPU)
14:59:06 [INFO] Started loading xlsx file ...
14:59:06 [INFO] Reading from file C:\Users\kpf\data\spr_plus\flaechen_rail_epanel.xlsx
14:59:06 [INFO] ... finished loading xlsx file in 0.02s (0.03s CPU)
14:59:06 [INFO] Started loading xlsx file ...
14:59:06 [INFO] Reading from file C:\Users\kpf\data\spr_plus\flaechen_rail_eboard.xlsx
14:59:06 [INFO] ... finished loading xlsx file in 0.01s (0.02s CPU)


In [53]:
spr_data_annotated = spr_data.merge(rail_displays, how="left", left_on="Flaeche_ID", right_on="Nummer")

In [55]:
with project_dir("axinova"):
    store_bin(spr_data_annotated, "spr_data.feather")

15:06:21 [INFO] Started storing feather file ...
15:06:21 [INFO] Writing to file C:\Users\kpf\data\axinova\spr_data.feather
15:06:21 [INFO] ... finished storing feather file in 0.3s (0.3s CPU)


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\kpf\\data\\axinova\\spr_data.feather'